In [34]:
import osmnx as ox
import momepy
import geopandas as gpd
import networkx
from osmnx import io
from geopandas import GeoDataFrame
from centerline.geometry import Centerline
path ='output'
project_crs = 'epsg:3857'

In [3]:
# Run only when download for first time
graph = ox.graph_from_place('Torino', network_type='all')
io.save_graph_geopackage(graph, filepath='.', encoding='utf-8', directed=False)


In [15]:
# preliminary cleaning
my_gdf = gpd.read_file(f'{path}/edges.shp')
not_null= my_gdf.dropna(subset='name')
not_null.to_file(f'{path}/not_null.shp')


In [19]:
not_null['tunnel'].unique()

array([None, 'building_passage', 'yes'], dtype=object)

In [20]:
not_roundaout = not_null[~((not_null['junction']=='roundabout')|(not_null['junction']=='circular')|(not_null['tunnel']=='building_passage')|(not_null['tunnel']=='yes'))]
not_roundaout.to_file(f'{path}/not_roundabout_tunnel.shp')

In [23]:
df_pro = not_roundaout.to_crs(project_crs)

In [61]:
streets_to_clear_segment = []
for group in df_pro.groupby('name'):
    res =group[1]
    buffer_test = res['geometry'].buffer(cap_style=3,distance=30,join_style=3)
    d =40/(2**0.5)
    res_translate = res.translate(d,d)
    new_data_0 = GeoDataFrame(geometry=res_translate,crs=project_crs).sjoin(GeoDataFrame(geometry=buffer_test,crs= project_crs),how='inner').reset_index()
    new_data = new_data_0[new_data_0['index']!= new_data_0['index_right']]
    if len(new_data) >0:
        print(f'{group[0]} has parallel segmetmts')
        streets_to_clear_segment.append(res)



18 has parallel segmetmts
Area pedonale Vittime dell'Immigrazione has parallel segmetmts
Autostrada Serenissima has parallel segmetmts
Ciclabile Carducci - Biglieri has parallel segmetmts
Corso Achille Mario Dogliotti has parallel segmetmts
Corso Adriatico has parallel segmetmts
Corso Alberto Picco has parallel segmetmts
Corso Alcide De Gasperi has parallel segmetmts
Corso Alessandro Tassoni has parallel segmetmts
Corso Appio Claudio has parallel segmetmts
Corso Belgio has parallel segmetmts
Corso Benedetto Brin has parallel segmetmts
Corso Benedetto Croce has parallel segmetmts
Corso Bernardino Telesio has parallel segmetmts
Corso Bolzano has parallel segmetmts
Corso Bramante has parallel segmetmts
Corso Brianza has parallel segmetmts
Corso Cadore has parallel segmetmts
Corso Caduti sul lavoro has parallel segmetmts
Corso Caio Plinio has parallel segmetmts
Corso Cairoli has parallel segmetmts
Corso Carlo Luigi Farini has parallel segmetmts
Corso Carlo Piaggia has parallel segmetmts
Co

In [57]:
my_groupby = df_pro.groupby('name')
res =my_groupby.get_group('11')
res.to_file(f'{path}/by_name.shp')
buffer_test = res['geometry'].buffer(cap_style=3,distance=30,join_style=3)
buffer_test.to_file(f'{path}/buffers_test.shp')
d =40/(2**0.5)
res_translate = res.translate(d,d)
res_translate.to_file(f'{path}/res_translate_test.shp')
new_data_0 = GeoDataFrame(geometry=res_translate,crs=project_crs).sjoin(GeoDataFrame(geometry=buffer_test,crs= project_crs),how='inner').reset_index()

In [58]:
new_data = new_data_0[new_data_0['index']!= new_data_0['index_right']]
new_data

,index,geometry,index_right


In [59]:
if len(new_data) >0:
    new_data.to_file(f'{path}/new_data.shp')
else:
    print('no parallel lines')

no parallel lines


In [53]:
new_data

,geometry,index_right
15436,"LINESTRING (857674.331 5626595.994, 857654.026...",15436


In [61]:
buffers = res_pro['geometry'].buffer(cap_style=3,distance=30,join_style=3)
buffers.to_file('buffers.shp')
one_buffer = buffers.unary_union
GeoDataFrame(geometry=list(one_buffer.geoms),crs='epsg:3857').to_file('one_buffer.shp')
my_centerline = Centerline(one_buffer)
GeoDataFrame(geometry=list(my_centerline.geoms),crs='epsg:3857').to_file('new_line.shp')

C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:34: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._input_geometry = input_geometry
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:35: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._interpolation_distance = abs(interpolation_distance)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:40: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._min_x, self._min_y = self._get_reduced_coordinates()


C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:34: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._input_geometry = input_geometry
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:35: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._interpolation_distance = abs(interpolation_distance)
C:\Users\Achituv\anaconda3\envs\momepy_env\lib\site-packages\centerline\geometry.py:40: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self._min_x, self._min_y = self._get_reduced_coordinates()


In [14]:
res.to_file('test.shp')